# Information Retrieval - Programming Assignment 4 - Training Learning-to-Rank (Using Programming Assignment 3 by Muhammad Falensi Azmi)

# Setup

In [4]:
!pip install python-terrier -q;
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git -q;
!pip install datasets;
!pip install gdown;
!pip install optuna


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -;

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: -;


In [1]:
import re
import pyterrier as pt
import pandas as pd
import numpy as np

from datasets import load_dataset
from pyterrier.measures import *

pt.init()  # This initializes PyTerrier

import os
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid
import pickle

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]
C:\Users\User\AppData\Local\Temp\ipykernel_3376\929666325.py:10: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()  # This initializes PyTerrier


# Data Preparation

In [2]:
# Load dataset
url = "https://drive.google.com/drive/folders/12AiGXUUlduwVJJrSOYp4RvLWNS8_X_xH?usp=drive_link"
gdown.download_folder(url, output=".", quiet=True)

['.\\corpus.csv',
 '.\\dev.csv',
 '.\\queries.csv',
 '.\\test.csv',
 '.\\train.csv']

In [2]:
collections = pd.read_csv("corpus.csv")
queries = pd.read_csv("queries.csv")

train_qrels = pd.read_csv("train.csv")
dev_qrels = pd.read_csv("dev.csv")
test_qrels = pd.read_csv("test.csv")

In [3]:
collections.head()

,id,text
0,49,"Colorâurine can be a variety of colors, most..."
1,12913,Rio de Janeiro: Annual Weather Averages. Febru...
2,14964,The judiciary (also known as the judicial syst...
3,17272,Painless swelling of the feet and ankles is a ...
4,18352,"Later that day, the National Hurricane Center ..."


In [4]:
queries.head()

,id,text
0,597651,what color is amber urine
1,88585,causes of swollen ankles and feet
2,508811,symptoms of strep throat for an adult
3,412886,is ilovemakonnen ovo
4,532152,uneven chest color


In [5]:
train_qrels.head()

,query-id,corpus-id,score
0,597651,49,1
1,88585,17272,1
2,508811,28092,1
3,412886,56458,1
4,532152,73237,1


## ✋ TODO

### Preprocessing 1: Change Column Name

In [6]:
# TODO: preprocess data according to needs
collections = collections.rename(columns={'id':'docno'})
collections.head()

,docno,text
0,49,"Colorâurine can be a variety of colors, most..."
1,12913,Rio de Janeiro: Annual Weather Averages. Febru...
2,14964,The judiciary (also known as the judicial syst...
3,17272,Painless swelling of the feet and ankles is a ...
4,18352,"Later that day, the National Hurricane Center ..."


In [7]:
queries = queries.rename(columns={'id':'qid', 'text':'query'})
queries.head()

,qid,query
0,597651,what color is amber urine
1,88585,causes of swollen ankles and feet
2,508811,symptoms of strep throat for an adult
3,412886,is ilovemakonnen ovo
4,532152,uneven chest color


In [8]:
train_qrels = train_qrels.rename(columns={'query-id':'qid', 'corpus-id':'docno', 'score':'label'})
train_qrels.head()

,qid,docno,label
0,597651,49,1
1,88585,17272,1
2,508811,28092,1
3,412886,56458,1
4,532152,73237,1


In [9]:
dev_qrels = dev_qrels.rename(columns={'query-id':'qid', 'corpus-id':'docno', 'score':'label'})
dev_qrels.head()

,qid,docno,label
0,174249,7067348,1
1,87892,7069601,1
2,264827,7071066,1
3,206117,7072160,1
4,196232,7072326,1


In [10]:
test_qrels = test_qrels.rename(columns={'query-id':'qid', 'corpus-id':'docno', 'score':'label'})
test_qrels.head()

,qid,docno,label
0,19335,1720387,0
1,19335,1871222,0
2,19335,1958102,0
3,19335,2046505,1
4,19335,2186129,0


### Preprocessing 2: convert label to binary

In [11]:
train_qrels.loc[train_qrels['label'] > 1, 'label'] = 1
dev_qrels.loc[dev_qrels['label'] > 1, 'label'] = 1
test_qrels.loc[test_qrels['label'] > 1, 'label'] = 1

In [12]:
print(train_qrels['label'].value_counts())
print(dev_qrels['label'].value_counts())
print(test_qrels['label'].value_counts())

label
1    5535
Name: count, dtype: int64
label
1    1005
Name: count, dtype: int64
label
0    566
1    470
Name: count, dtype: int64


### Preprocessing 3: remove nonalphanumerics and lowercasting

In [13]:
def remove_nonalphanum(text):
  pattern = re.compile('[\W_]+')
  return pattern.sub(' ', text)

def lowercast(text):
  return text.lower()

In [14]:
collections['text'] = collections['text'].apply(remove_nonalphanum)
collections['text'] = collections['text'].apply(lowercast)
collections.head()

,docno,text
0,49,colorâ urine can be a variety of colors most o...
1,12913,rio de janeiro annual weather averages februar...
2,14964,the judiciary also known as the judicial syste...
3,17272,painless swelling of the feet and ankles is a ...
4,18352,later that day the national hurricane center n...


In [15]:
queries['query'] = queries['query'].apply(remove_nonalphanum)
queries['query'] = queries['query'].apply(lowercast)
queries.head()

,qid,query
0,597651,what color is amber urine
1,88585,causes of swollen ankles and feet
2,508811,symptoms of strep throat for an adult
3,412886,is ilovemakonnen ovo
4,532152,uneven chest color


### Preprocessing 4: Convert docno and qid to string

In [16]:
collections["docno"] = collections["docno"].astype(str)
queries["qid"] = queries["qid"].astype(str)

train_qrels["qid"] = train_qrels["qid"].astype(str)
dev_qrels["qid"] = dev_qrels["qid"].astype(str)
test_qrels["qid"] = test_qrels["qid"].astype(str)

train_qrels["docno"] = train_qrels["docno"].astype(str)
dev_qrels["docno"] = dev_qrels["docno"].astype(str)
test_qrels["docno"] = test_qrels["docno"].astype(str)

### Preprocessing 5: Stemming and Stopwords Removal (done in the `Indexing` code)

### Preprocessing 6: Limit num of queries based on certain rule.

In [17]:
print(train_qrels.shape)
print(dev_qrels.shape)
print(test_qrels.shape)
print(queries.shape)

(5535, 3)
(1005, 3)
(1036, 3)
(6577, 2)


In [18]:
# Verify overlap between queries and qrels
common_train_qids = set(queries['qid']) & set(train_qrels['qid'])
common_val_qids = set(queries['qid']) & set(dev_qrels['qid'])
common_test_qids = set(queries['qid']) & set(test_qrels['qid'])

# # Filter queries to ensure overlap
train_queries = queries[queries['qid'].isin(common_train_qids)]
val_queries = queries[queries['qid'].isin(common_val_qids)]
test_queries = queries[queries['qid'].isin(common_test_qids)]

# Further sampling if needed for efficiency (NOT USED)
train_sample = train_queries
# .sample(n=min(4000, len(train_queries)), random_state=42)
val_sample = val_queries
# .sample(n=min(1000, len(val_queries)), random_state=42)
test_sample = test_queries
# .sample(n=min(250, len(test_queries)), random_state=42)

In [19]:
print(train_queries.shape, val_queries.shape, test_queries.shape)
print(train_sample.shape, val_sample.shape, test_sample.shape)
print(train_qrels.shape, dev_qrels.shape, test_qrels.shape)

(5533, 2) (1001, 2) (43, 2)
(5533, 2) (1001, 2) (43, 2)
(5535, 3) (1005, 3) (1036, 3)


In [20]:
train_topics, dev_topics, test_topics = train_sample, val_sample, test_sample

# Indexing

## ✋ TODO

In [21]:
index_dir = os.path.join(os.getcwd(), "index")

In [22]:
# TODO: adjust indexer

pd_indexer = pt.terrier.DFIndexer(index_dir, \
                          type = pt.index.IndexingType.CLASSIC, \
                          tokeniser = pt.index.TerrierTokeniser('utf'), \
                          stemmer = pt.TerrierStemmer.porter, \
                          stopwords = pt.TerrierStopwords.terrier, \
                          blocks = True, \
                          verbose = True)

index_ref = pd_indexer.index(collections["text"], collections)

C:\Users\User\AppData\Local\Temp\ipykernel_3376\1839210181.py:3: DeprecationWarning: Call to deprecated class DFIndexer. (use pt.terrier.IterDictIndexer().index(dataframe.to_dict(orient='records')) instead) -- Deprecated since version 0.11.0.
  pd_indexer = pt.terrier.DFIndexer(index_dir, \
  0%|          | 0/6992 [00:00<?, ?documents/s]

100%|██████████| 6992/6992 [00:06<00:00, 1087.71documents/s]


In [23]:
index_ref = pt.IndexFactory.of(index_dir)

# Baseline

In [25]:
# Set K=30 sebagai hasil akhir setelah re-rank (SERP)
K = 30

In [28]:
bm25 = pt.terrier.Retriever(index_ref, wmodel = "BM25")

In [29]:
experiment_raw_bm25_dir = os.path.join(os.getcwd(), "result")
result = pt.Experiment(
    [bm25 % K], # TOP K only to make it comparable
    topics=test_topics,
    qrels=test_qrels,
    eval_metrics=["P_30","map","recip_rank","ndcg"],
    names=["BM25"],
    save_dir=experiment_raw_bm25_dir,
    save_mode='overwrite',
    verbose=True,
)

pt.Experiment: 100%|██████████| 1/1 [00:00<00:00,  1.24system/s]


In [30]:
result

,name,P_30,map,recip_rank,ndcg
0,BM25,0.270543,0.542489,0.723107,0.691632


# Re-rank

## Feature

### ✋ TODO

In [31]:
# TODO: propose and implement features

# Models (other than BM25)
tfidf = pt.terrier.Retriever(index_ref, wmodel="TF_IDF")
pl2 = pt.terrier.Retriever(index_ref, wmodel="PL2")
bb2 = pt.terrier.Retriever(index_ref, wmodel="BB2")

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(collections["text"])

def cosine_similarity_feature(doc, query):
    doc_vector = tfidf_vectorizer.transform([doc])
    query_vector = tfidf_vectorizer.transform([query])
    return cosine_similarity(doc_vector, query_vector)[0][0]

def length_ratio_feature(doc, query):
    doc_length = len(doc.split())
    query_length = len(query.split())
    return doc_length / query_length if query_length > 0 else 0

# generate feature
def generate_features(doc, query):
    features = [
        cosine_similarity_feature(doc, query),
        length_ratio_feature(doc, query)
    ]
    return np.array(features)

features = pt.apply.doc_features(lambda row: generate_features(row["text"], row["query"]))

In [32]:
# Get "text" column using `pt.text.get_text(index_ref, "text")`
pipeline = bm25 >> pt.text.get_text(index_ref, "text") >> (features ** bm25 ** tfidf ** pl2 ** bb2)

## Learning-to-Rank

### ✋ TODO

In [35]:
# TODO: perform hyperparameter tuning on the LambdaMART

# Below hyperparameters are taken from the example on PyTerrier documentation:
# https://pyterrier.readthedocs.io/en/latest/ltr.html#gradient-boosted-trees-lambdamart

# XGBoost hyperparameters documentation:
# https://xgboost.readthedocs.io/en/stable/parameter.html

# test using base model
lmart_model = xgb.sklearn.XGBRanker(objective = 'rank:ndcg',
                                    max_depth = '3',
                                    subsample = '0.6',
                                    gamma = '1.0',
                                    verbose = 1,
                                    random_state = 42,
                                    n_jobs=-1,
                                    )

In [36]:
lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_model, form = "ltr")
lmart_pipe.fit(
    train_topics, 
    train_qrels, 
    dev_topics, 
    dev_qrels)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:158: UserWarning: [19:00:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


In [41]:
with open('lmart_model.pkl', 'wb') as file:
    pickle.dump(lmart_model, file)

## Hyperparameter Tuning

In [82]:
import time

# Define the parameter grid for Grid Search
param_grid = {
    # Learning Parameters
    'subsample' : [0.4, 0.6],
    'gamma': [0.5, 1.0],
    'max_depth': [3]
}

# Initialize variables to store the best model and score
best_model = None
best_score = -1
best_params = None

now = time.time()

# Perform Grid Search
for params in ParameterGrid(param_grid):
    print(f"Testing parameters: {params}")
    lmart_model = xgb.sklearn.XGBRanker(
        objective='rank:ndcg',
        random_state=42,
        n_jobs=-1,
        verbose=2,
        device='cuda',
        **params
    )
    lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_model, form="ltr")
    lmart_pipe.fit(train_topics, train_qrels, dev_topics, dev_qrels)
    
    # Evaluate on validation set
    eval_results = pt.Experiment([lmart_pipe % K], dev_topics, dev_qrels, eval_metrics=["ndcg"], names=["L2R"])
    score = eval_results["ndcg"][0]
    
    # Update the best model if current is better
    if score > best_score:
        best_score = score
        best_model = lmart_pipe
        best_params = params

    print(f"Score: {score}. Best is {best_score} with params {best_params}")
    print(f"time:{int(time.time()-now)}")

Testing parameters: {'gamma': 0.5, 'max_depth': 3, 'subsample': 0.4}


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [05:33:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Score: 0.8955371784223587. Best is 0.8955371784223587 with params {'gamma': 0.5, 'max_depth': 3, 'subsample': 0.4}
time:6005
Testing parameters: {'gamma': 0.5, 'max_depth': 3, 'subsample': 0.6}


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [07:14:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Score: 0.8972169310694136. Best is 0.8972169310694136 with params {'gamma': 0.5, 'max_depth': 3, 'subsample': 0.6}
time:12074
Testing parameters: {'gamma': 1.0, 'max_depth': 3, 'subsample': 0.4}


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [08:58:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Score: 0.8940310131488552. Best is 0.8972169310694136 with params {'gamma': 0.5, 'max_depth': 3, 'subsample': 0.6}
time:18305
Testing parameters: {'gamma': 1.0, 'max_depth': 3, 'subsample': 0.6}


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [10:38:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Score: 0.8974502623816912. Best is 0.8974502623816912 with params {'gamma': 1.0, 'max_depth': 3, 'subsample': 0.6}
time:24342


In [125]:
print(best_params, best_score)

{'gamma': 1.0, 'max_depth': 3, 'subsample': 0.6} 0.8974502623816912


### Optimization using Optuna (not used, sebagai dokumentasi saja)

In [ ]:
# import optuna

# now = time.time()
# def objective(trial):
#     params = {
#         'gamma': trial.suggest_float("gamma", 0.0, 0.1),
#         'max_depth': trial.suggest_int("max_depth", 3, 4),
#         'subsample': trial.suggest_float("subsample", 0.4, 0.6),
#         'colsample_bytree': trial.suggest_float("colsample_bytree", 0.6, 1.0),
#         'lambda': trial.suggest_float("lambda", 0.8, 1.2),
#         'alpha': trial.suggest_float("alpha", 0.8, 1.2),
#     }
    
#     lmart_model = xgb.sklearn.XGBRanker(
#         objective='rank:ndcg',
#         random_state=42,
#         n_jobs=-1,
#         verbose=1,
#         **params
#     )
#     lmart_pipe = pipeline >> pt.ltr.apply_learned_model(lmart_model, form="ltr")
#     lmart_pipe.fit(train_topics, train_qrels, dev_topics, dev_qrels)
    
#     eval_results = pt.Experiment([lmart_pipe % K], dev_topics, dev_qrels, eval_metrics=["ndcg"], names=["L2R"])
#     score = eval_results["ndcg"][0]
    
#     print(f"time:{int(time.time()-now)}")
#     return score

# # Create an Optuna study
# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=50)

# # Get the best parameters and score
# best_params = study.best_params
# best_score = study.best_value
# print(f"Best parameters: {best_params}")
# print(f"Best score: {best_score}")

[I 2024-11-25 16:06:07,447] A new study created in memory with name: no-name-bf8aaefd-ddc4-48e8-9791-44e48c57b3bb
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:10:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-11-25 16:11:15,950] Trial 0 finished with value: 0.889780712576313 and parameters: {'gamma': 0.07529041863109562, 'max_depth': 3, 'subsample': 0.4866770297338993, 'colsample_bytree': 0.7678007085418099, 'lambda': 1.1341187803517139, 'alpha': 0.9390619151576209}. Best is trial 0 with value: 0.889780712576313.


time:308


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:15:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-11-25 16:16:13,960] Trial 1 finished with value: 0.8820193322165437 and parameters: {'gamma': 0.09403365053682722, 'max_depth': 4, 'subsample': 0.597264458515771, 'colsample_bytree': 0.9397230791512087, 'lambda': 0.871818856218183, 'alpha': 0.8732424380900944}. Best is trial 0 with value: 0.889780712576313.


time:606


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:20:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-11-25 16:21:20,039] Trial 2 finished with value: 0.8935956963698062 and parameters: {'gamma': 0.049541708381048245, 'max_depth': 3, 'subsample': 0.4124214980744642, 'colsample_bytree': 0.7282303118311557, 'lambda': 0.9619513230536485, 'alpha': 0.8791045324846161}. Best is trial 2 with value: 0.8935956963698062.


...

time:13588


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:56:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-11-25 19:57:15,129] Trial 48 finished with value: 0.8858792108523451 and parameters: {'gamma': 0.047747525854816675, 'max_depth': 3, 'subsample': 0.40057850672414885, 'colsample_bytree': 0.8893331495547188, 'lambda': 1.0086463759089521, 'alpha': 0.8772117953431369}. Best is trial 2 with value: 0.8935956963698062.


time:13867


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [20:01:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-11-25 20:02:01,079] Trial 49 finished with value: 0.8824619835829111 and parameters: {'gamma': 0.08397478809546507, 'max_depth': 4, 'subsample': 0.45380642565779505, 'colsample_bytree': 0.6533208035238253, 'lambda': 0.8807013227732342, 'alpha': 1.1342702853357063}. Best is trial 2 with value: 0.8935956963698062.


time:14153
Best parameters: {'gamma': 0.049541708381048245, 'max_depth': 3, 'subsample': 0.4124214980744642, 'colsample_bytree': 0.7282303118311557, 'lambda': 0.9619513230536485, 'alpha': 0.8791045324846161}
Best score: 0.8935956963698062


# Evaluation


## ✋ TODO

In [43]:
final_eval_result_dir = os.path.join(os.getcwd(), "result_final")
eval_results = pt.Experiment([bm25 % K, lmart_pipe % K],
                            topics=test_queries,
                            qrels=test_qrels,
                            eval_metrics=["P_30","map","recip_rank","ndcg"],
                            names=["BM25", "L2R"],
                            baseline=0,
                            save_dir=final_eval_result_dir,
                            save_mode="overwrite",
                            verbose=True,
                            )

pt.Experiment: 100%|██████████| 2/2 [00:28<00:00, 14.09s/system]


In [44]:
eval_results

,name,map,recip_rank,P_30,ndcg,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P_30 +,P_30 -,P_30 p-value,ndcg +,ndcg -,ndcg p-value
0,BM25,0.542489,0.723107,0.270543,0.691632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L2R,0.559806,0.741711,0.279070,0.709557,23.0,15.0,0.613267,7.0,5.0,0.648245,5.0,1.0,0.124827,24.0,14.0,0.505503


In [48]:
with open('lmart_model_tp4.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [49]:
pipe_new = pipeline >> pt.ltr.apply_learned_model(loaded_model, form = "ltr")

In [50]:
final_eval_result_dir = os.path.join(os.getcwd(), "result_final")
eval_results = pt.Experiment([bm25 % K, pipe_new % K],
                            topics=test_queries,
                            qrels=test_qrels,
                            eval_metrics=["P_30","map","recip_rank","ndcg"],
                            names=["BM25", "L2R"],
                            baseline=0,
                            save_dir=final_eval_result_dir,
                            save_mode="overwrite",
                            verbose=True,
                            )
eval_results

pt.Experiment: 100%|██████████| 2/2 [00:30<00:00, 15.30s/system]


,name,map,recip_rank,P_30,ndcg,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P_30 +,P_30 -,P_30 p-value,ndcg +,ndcg -,ndcg p-value
0,BM25,0.542489,0.723107,0.270543,0.691632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,L2R,0.559806,0.741711,0.279070,0.709557,23.0,15.0,0.613267,7.0,5.0,0.648245,5.0,1.0,0.124827,24.0,14.0,0.505503
